In [1]:
from sentence_transformers import CrossEncoder

c:\Users\kryst\.virtualenvs\lecture_search-VJxE5dGJ\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.llms import LlamaCpp
llm = LlamaCpp(model_path=r"C:\Users\kryst\Documents\models\llm\ggml-vic7b-q5_0.bin", n_ctx=1024, temperature=0.6)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [3]:
def get_prompt(term, passages, other_terms):
    context = "\n".join(passages)
    prompt = f"""The term "{term}" is new and has never been used before. The only information about this new term "{term}" that you have is in these three separate passages:\n {context}
Task: Using only the information from the passages, write a short and precise definition of "{term}". Omit facts that do not appear in the passages. You have already defined these terms and may use them: {[t for t in other_terms]}.
The definition of nadir points is:"""
    return prompt


In [4]:
import lecture_search.app.cozo_client as cozo_db

client = cozo_db.get_client(r"C:\Users\kryst\Documents\Artificial Intelligence\Artificial Intelligence - sem6\courses\cozo_test.db")

In [5]:
## Load our cross-encoder. Use fast tokenizer to speed up the tokenization
model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2')
def rank_passages(query, passages, k =3):

    #Concatenate the query and all passages and predict the scores for the pairs [query, passage]
    model_inputs = [[query, passage] for passage in passages]
    scores = model.predict(model_inputs)

    #Sort the scores in decreasing order
    results = [{'input': inp, 'score': score} for inp, score in zip(model_inputs, scores)]
    results = sorted(results, key=lambda x: x['score'], reverse=True)

    print("Query:", query)
    return [result["input"][1] for result in results[:k]]

In [6]:
def get_passages(phrase):
    query = f"what is the definition of {phrase}"
    results = cozo_db.semantic_search(client,query, cozo_db.get_bi_encoder())
    results = rank_passages(query, [result["sentence"] for result in results[:3]])
    return results

get_passages("nadir points")

Query: what is the definition of nadir points


['Therefore, it is often used as an estimate of the nadir point. Also, for numerical reasons, the ideal point is often not used directly. In turn, it is transformed into a utopian point with marginally better objective values on all objectives.',
 ' Since, in practice, the entire Pareto front is unknown, the nadir point can only be approximated. It is much easier to determine the max point with the worst objective values of the entire objective space. Therefore, it is often used as an estimate of the nadir point.',
 'It represents the best of each objective function, and therefore it is typically non-existent or infeasible. In turn, the nadir objective vector is the upper bound of the Pareto front, representing the worst of all objective functions but with values delimited to the Pareto optimal solutions. Typically, it is a dominated point.']

In [7]:
terms = [] # normally load them from extract_phrases script

def find_defining_phrases(term, passages):
    terms_for_definition = []
    for f_passage in passages:
        for p in terms:
            if p in f_passage and p != term:
                terms_for_definition.append(p)
    return set(terms_for_definition)


In [8]:
term = "nadir points"
passages = get_passages(term)
# other_terms = find_defining_phrases(term, passages)
other_terms = {'utopian point', 'Pareto optimal', 'Pareto front', 'nadir objective vector', 'objective function', 'objective vector', 'optimal solution', 'ideal point'} 
prompt = get_prompt(term, passages, other_terms)
print(prompt)

Query: what is the definition of nadir points
The term "nadir points" is new and has never been used before. The only information about this new term "nadir points" that you have is in these three separate passages:
 Therefore, it is often used as an estimate of the nadir point. Also, for numerical reasons, the ideal point is often not used directly. In turn, it is transformed into a utopian point with marginally better objective values on all objectives.
 Since, in practice, the entire Pareto front is unknown, the nadir point can only be approximated. It is much easier to determine the max point with the worst objective values of the entire objective space. Therefore, it is often used as an estimate of the nadir point.
It represents the best of each objective function, and therefore it is typically non-existent or infeasible. In turn, the nadir objective vector is the upper bound of the Pareto front, representing the worst of all objective functions but with values delimited to the Pa

In [9]:
llm(prompt)